### **DAY 6 (14/01/26) – Medallion Architecture**

### Learn:

- Bronze (raw) → Silver (cleaned) → Gold (aggregated)
- Best practices for each layer
- Incremental processing patterns

### 🛠️ Tasks:

1. Design 3-layer architecture
2. Build Bronze: raw ingestion
3. Build Silver: cleaning & validation
4. Build Gold: business aggregates

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
# BRONZE: Raw ingestion
raw = spark.read.format('csv').load('/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv', header=True, inferSchema=True)

raw.withColumn("ingestion_ts", F.current_timestamp())\
    .write.format("delta").mode("overwrite").save('/Volumes/workspace/ecommerce/ecommerce_data/delta/bronze/events')

In [0]:
# SILVER: Cleaned data

bronze = spark.read.format('delta').load('/Volumes/workspace/ecommerce/ecommerce_data/delta/bronze/events')
silver = bronze.filter(F.col('price')>0)\
        .filter(F.col("price") < 10000)\
        .dropDuplicates(['user_session','event_time'])\
        .withColumn("event_date",F.to_date('event_time'))\
        .withColumn("price_tier",
                    F.when(F.col('price') <10,'Budget')\
                     .when(F.col('price') <50,'Mid')\
                     .otherwise('Premium'))
silver.write.format('delta').mode("overwrite").save('/Volumes/workspace/ecommerce/ecommerce_data/delta/silver/events')


In [0]:
# GOLD: Aggregates
silver = spark.read.format('delta').load('/Volumes/workspace/ecommerce/ecommerce_data/delta/silver/events')
product_perf =  silver.groupBy('brand')\
                .agg(
                    F.countDistinct(F.when(F.col('event_type')=='view',F.col('user_id'))).alias('views')\
                    ,F.countDistinct(F.when(F.col('event_type')=='purchase',F.col('user_id'))).alias('purchases')\
                    ,F.avg(F.when(F.col('event_type')=='purchase',F.col('price'))).alias('avg_revenue')
                    )\
                .withColumn('Conversion_rate',F.col('purchases')/F.col('views')*100)
product_perf.write.format('delta').mode("overwrite").save('/Volumes/workspace/ecommerce/ecommerce_data/delta/gold/products')